In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# -----------------------------
# Date range: last 10 years
# -----------------------------
end_date = datetime.today()
start_date = end_date - timedelta(days=10 * 365)

# -----------------------------
# All tickers (flattened)
# -----------------------------
tickers = [
    # Communication Services
    "CHTR", "CMCSA", "DIS", "EA", "GOOGL",
    # Consumer Discretionary
    "AMZN", "APTV", "AZO", "BBWI", "BBY",
    # Consumer Staples
    "ADM", "BG", "CAG", "CHD", "CL",
    # Energy
    "XOM", "CVX", "COP", "EOG", "SLB",
    # Financials
    "ACGL", "AFL", "AIG", "AIZ", "AJG",
    # Health Care
    "A", "ABBV", "ABT", "AMGN", "BAX",
    # Industrials
    "AAL", "ADP", "ALLE", "AME", "AOS",
    # Information Technology
    "AAPL", "ACN", "ADBE", "ADI", "ADSK",
    # Materials
    "ALB", "APD", "AVY", "BALL", "CE",
    # Real Estate
    "AMT", "ARE", "AVB", "BXP", "CBRE",
    # Utilities
    "AEE", "AEP", "AES", "ATO", "AWK"
]

# -----------------------------
# Download ALL tickers at once
# -----------------------------
print("Downloading data from Yahoo Finance...")

data = yf.download(
    tickers,
    start=start_date.strftime("%Y-%m-%d"),
    end=end_date.strftime("%Y-%m-%d"),
    auto_adjust=False,
    progress=True,
)

# -----------------------------
# Keep only Adjusted Close
# -----------------------------
adj_close = data["Adj Close"]

# -----------------------------
# Clean & save
# -----------------------------
adj_close = adj_close.sort_index()
adj_close.reset_index(inplace=True)

adj_close.to_csv("yahoo_all_stocks_10y_merged.csv", index=False)

print("✅ Saved to yahoo_all_stocks_10y_merged.csv")


[****                   9%                       ]  5 of 55 completed

[*********************100%***********************]  55 of 55 completed

✅ Saved to yahoo_all_stocks_10y_merged.csv


In [12]:

import pandas as pd
from pathlib import Path

INPUT_FILE = "sp500_esg_data.csv"
OUTPUT_FILE = "selected_esg_scores.csv"

# 1) Load the master ESG table
df = pd.read_csv(INPUT_FILE)

# 2) Your categories and tickers
groups = {
    "Communication Services": ["CHTR", "CMCSA", "DIS", "EA", "GOOGL"],
    "Consumer Discretionary": ["AMZN", "APTV", "AZO", "BBWI", "BBY"],
    "Consumer Staples": ["ADM", "BG", "CAG", "CHD", "CL"],
    "Energy": ["XOM", "CVX", "COP", "EOG", "SLB"],
    "Financials": ["ACGL", "AFL", "AIG", "AIZ", "AJG"],
    "Health Care": ["A", "ABBV", "ABT", "AMGN", "BAX"],
    "Industrials": ["AAL", "ADP", "ALLE", "AME", "AOS"],
    "Information Technology": ["AAPL", "ACN", "ADBE", "ADI", "ADSK"],
    "Materials": ["ALB", "APD", "AVY", "BALL", "CE"],
    "Real Estate": ["AMT", "ARE", "AVB", "BXP", "CBRE"],
    "Utilities": ["AEE", "AEP", "AES", "ATO", "AWK"],
}

# 3) Build a ticker -> category map
ticker_to_category = {t: cat for cat, lst in groups.items() for t in lst}
wanted = set(ticker_to_category.keys())

# 4) Normalize tickers in your DataFrame (uppercase, strip)
df["Symbol"] = df["Symbol"].astype(str).str.strip().str.upper()

# 5) Filter only the tickers you want
sel = df[df["Symbol"].isin(wanted)].copy()

# 6) Add the Category from your mapping
sel["Category"] = sel["Symbol"].map(ticker_to_category)

# 7) Select and rename columns; `totalEsg` is your ESG score
out = (
    sel[["Symbol", "Category", "totalEsg"]]
    .rename(columns={"Symbol": "Ticker", "totalEsg": "ESG_Score"})
    .sort_values(["Category", "Ticker"])
    .reset_index(drop=True)
)

# 8) Save to CSV
Path(OUTPUT_FILE).write_text(out.to_csv(index=False))
print(f"Saved {OUTPUT_FILE} with {len(out)} rows")


Saved selected_esg_scores.csv with 55 rows
